In [9]:
import torch
import numpy as np

import dataloader

import os
import subprocess
import socket

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"


In [19]:
import pickle
file = open("/jet/home/billyli/data_folder/DayLongAudio/audioset_py3/billyli_reproduce_code/video_stats.pkl",'rb')
object_file = pickle.load(file)
file.close()
print(object_file)
print(object_file['mean'].shape)
print(object_file['variance'].shape)

{'mean': array([[0.39146292, 0.48243216, 0.42972042, ..., 0.68159719, 0.62372522,
        0.51758939],
       [0.3944056 , 0.4850449 , 0.4317698 , ..., 0.68096074, 0.62257115,
        0.51601626],
       [0.39570916, 0.48641873, 0.43300768, ..., 0.68020981, 0.6216692 ,
        0.5155688 ],
       ...,
       [0.39344822, 0.48339734, 0.43000543, ..., 0.68250182, 0.62308511,
        0.51609492],
       [0.39384353, 0.48347628, 0.42965156, ..., 0.68269066, 0.62396854,
        0.51608554],
       [0.39380813, 0.48347781, 0.42973174, ..., 0.68226304, 0.62348332,
        0.5158906 ]]), 'variance': array([[0.17720908, 0.29149326, 0.25373703, ..., 0.35772536, 0.26358646,
        0.11088838],
       [0.17850959, 0.29320565, 0.25563043, ..., 0.35843417, 0.26304044,
        0.1104022 ],
       [0.17970829, 0.29485482, 0.25715749, ..., 0.358018  , 0.26269602,
        0.11042276],
       ...,
       [0.17725859, 0.29040507, 0.25324999, ..., 0.36093449, 0.26408747,
        0.11042896],
       [0.178

In [10]:

curr_node = socket.gethostname().split('.')[0]
batcmd=f"squeue -u billyli | grep {curr_node}"
curr_slurm = subprocess.check_output(batcmd, shell=True, text=True)
slurm_id = 12913345#curr_slurm.strip().split(' ')[0]
local = f"/local/slurm-{slurm_id}/local/audio/data"

In [11]:
print(local)

/local/slurm-12913345/local/audio/data


# set skip_norm as True only when you are computing the normalization stats

In [12]:
audio_conf = {'num_mel_bins': 64, 'target_length': 400, 'freqm': 36, 'timem': 192, 'mixup': 0.3, 'skip_norm': True, 'mode': 'train', 'dataset': 'audioset'}
train_loader = torch.utils.data.DataLoader(dataloader.AudiosetDataset(os.path.join(local,'datafiles/audioset_bal_train_data_v.json'), label_csv=os.path.join(local, 'class_labels_indices.csv'),
                                audio_conf=audio_conf), batch_size=100, shuffle=False, num_workers=8, pin_memory=True)

---------------the train dataloader---------------
now using following mask: 36 freq, 192 time
now using mix-up with rate 0.300000
now process audioset
now skip normalization (use it ONLY when you are computing the normalization stats).
number of classes is 527


In [13]:
#128 x 1024 features:
# freqm 48 timem 192 mixup 0.5: -3.3458831 4.1563106
# freqm 36 timem 192 mixup 0.5: -3.5172966 4.1748486
# freqm 36 timem 192 mixup 0.3: -3.6788461 4.3739743

#64 x 400 features:
# freqm 24 timem 75 mixup 0.5: -24.584522, 37.10214
# freqm 12 timem 75 mixup 0.5: -27.248297 37.681698
# freqm 12 timem 75 mixup 0.3: -29.072655 40.71728
# freqm 12 timem 60 mixup 0.5: -27.712713 37.934616
# freqm 12 timem 60 mixup 0.3: -29.686901 40.898224
# freqm 12 timem 60 mixup 0.2: -30.733978 42.267876
# freqm 12 timem 56 mixup 0.4: -28.799559 39.34941
# freqm 12 timem 56 mixup 0.25: -30.333525 41.43794
# freqm 10 timem 50 mixup 0.3: -30.68206 41.202892
# freqm 10 timem 50 mixup 0.2: -31.451344 42.354965
# freqm 6 timem 25 mixup 0.5: -30.446014 38.751587
# freqm 6 timem 25 mixup 0.3: -32.60682 41.53026
# freqm 6 timem 25 mixup 0.2: -33.632145 42.845398
# freqm 6 timem 25 mixup 0.1: -34.673824 44.153343
# freqm 0 timem 0 mixup 0: -38.193954 45.802143


In [14]:

mean=[]
std=[]
for i, (audio_input, labels, video_input) in enumerate(train_loader):
    cur_mean = torch.mean(video_input)
    cur_std = torch.std(video_input)
    mean.append(cur_mean)
    std.append(cur_std)
    print(i, cur_mean, cur_std)
print(np.mean(mean), np.mean(std))

0 tensor(0.5117) tensor(0.4851)
1 tensor(0.5125) tensor(0.4961)
2 tensor(0.5157) tensor(0.4966)
3 tensor(0.5111) tensor(0.4958)
4 tensor(0.5053) tensor(0.4877)
5 tensor(0.5138) tensor(0.4887)
6 tensor(0.5127) tensor(0.4865)
7 tensor(0.5364) tensor(0.5353)
8 tensor(0.5083) tensor(0.4935)
9 tensor(0.5146) tensor(0.4901)
10 tensor(0.5047) tensor(0.4708)
11 tensor(0.5207) tensor(0.5204)
12 tensor(0.5105) tensor(0.4915)
13 tensor(0.5102) tensor(0.4925)
14 tensor(0.5122) tensor(0.4939)
15 tensor(0.5075) tensor(0.4777)
16 tensor(0.5213) tensor(0.5227)
17 tensor(0.5074) tensor(0.4811)
18 tensor(0.5055) tensor(0.4899)
19 tensor(0.5150) tensor(0.4969)
20 tensor(0.5176) tensor(0.5157)
21 tensor(0.4968) tensor(0.4764)
22 tensor(0.5154) tensor(0.4995)
23 tensor(0.5099) tensor(0.4875)
24 tensor(0.5101) tensor(0.5015)
25 tensor(0.5095) tensor(0.4900)
26 tensor(0.5010) tensor(0.4719)
27 tensor(0.5082) tensor(0.5031)
28 tensor(0.5243) tensor(0.5446)
29 tensor(0.5035) tensor(0.4858)
30 tensor(0.5139) te